In [ ]:
import sys
!{sys.executable} -m pip install geopandas

In [67]:
import json
import time
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import os
import geojson
from pathlib import Path
import osmnx as ox
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from scipy import spatial
from scipy.spatial import KDTree

cwd = Path().resolve()
boundary_geojson = gpd.read_file(os.path.join(Path(cwd).parent, 'data', 'geojson', 'vienna.geojson'))
boundary_geojson.drop(columns=['cartodb_id', 'created_at', 'updated_at'], inplace=True)
region = boundary_geojson.geometry.unary_union

def get_local_crs(y, x):
    x = ox.utils_geo.bbox_from_point((y, x), dist = 500, project_utm = True, return_crs = True)
    return x[-1]
  
# Set longitude and latitude of Vienna
lon_latitude = 48.210033
lon_longitude = 16.363449

local_utm_crs = get_local_crs(lon_latitude, lon_longitude)
#https://nominatim.openstreetmap.org/search?q=Universit%C3%A4tsring+2,%201010+Wien&format=json&polygon=1&addressdetails=1

In [68]:
def get_lat_long(point):
    """ get latitude and longitude coordinate from POINT geometry """
    try:
        return pd.Series([point.x, point.y])
    except:
        pass

def geo_coordinates(df):
    """ import from csv in geopandas dataframe
    source: https://stackoverflow.com/questions/61122875/geopandas-how-to-read-a-csv-and-convert-to-a-geopandas-dataframe-with-polygons
    """
    df['geometry'] = df['geometry'].apply(lambda x: x.centroid if type(x) == Polygon else (x.centroid if type(x) == MultiPolygon else x))
    df[['long', 'lat']] = df.apply(lambda x: get_lat_long(x['geometry']), axis=1)
    df = df[df['geometry'].apply(lambda x : x.type=='Point' )]
    df = df.to_crs(local_utm_crs)
    return df

In [69]:
from shapely import wkt

def import_csv_to_gpd(name):
    """ import the csv file a gepandas dataframe """
    df = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'osm', f'{name}.csv'), sep=",")
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
    return gdf

restaurant = import_csv_to_gpd('restaurant')
cafe = import_csv_to_gpd('cafe')
attraction = import_csv_to_gpd('attraction')
station = import_csv_to_gpd('attraction')
bar = import_csv_to_gpd('bar')
biergarten = import_csv_to_gpd('biergarten')
fast_food = import_csv_to_gpd('fast_food')
pub = import_csv_to_gpd('pub')
nightclub = import_csv_to_gpd('nightclub')
theatre= import_csv_to_gpd('theatre')
university= import_csv_to_gpd('university')
attraction= import_csv_to_gpd('attraction')

restaurant = geo_coordinates(restaurant)
cafe = geo_coordinates(cafe)
bar = geo_coordinates(bar)
station = geo_coordinates(station)
biergarten = geo_coordinates(biergarten)
fast_food = geo_coordinates(fast_food)
pub = geo_coordinates(pub)
nightclub = geo_coordinates(nightclub)
theatre = geo_coordinates(theatre)
university = geo_coordinates(university)
attraction = geo_coordinates(attraction)


C:\Users\arlin\AppData\Local\Temp\ipykernel_1944\2915294209.py:5: DtypeWarning: Columns (26,45,46,47,50,54,59,60,61,65,67,69,70,71,72,77,79,83,85,86,87,88,89,90,94,95,96,100,101,102,103,110,111,112,114,117,123,124,125,128,129,130,131,132,134,135,136,139,140,141,145,147,148,149,150,151,155,157,161,162,163,165,166,167,170,171,172,173,175,176,177,179,180,181,183,184,189,191,193,195,197,199,200,202,203,204,205,206,208,210,211,212,213,214,215,216,217,218,219,220,221,222,223,225,226,228,229,230,231,232,233,234,237,240,241,242,243,244,245,246,247,248,249,250,251,252,254,255,256) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(Path(cwd).parent, 'data', 'osm', f'{name}.csv'), sep=",")


In [70]:
restaurant.head()

,element_type,osmid,addr:city,addr:country,addr:housenumber,addr:postcode,addr:street,amenity,contact:website,cuisine,...,location,origin,source:geometry,baby_feeding,changing_table:count,fee,ways,type,long,lat
0,node,46823888,Wien,AT,38,1150.0,Selzergasse,restaurant,http://www.heidingers.at,regional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.323003,48.198231
1,node,46823910,Wien,AT,13,1150.0,Meiselstraße,restaurant,NaN,pizza,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.320545,48.197481
2,node,46823962,NaN,NaN,NaN,NaN,NaN,restaurant,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.320848,48.199091
3,node,55411636,NaN,NaN,NaN,NaN,NaN,restaurant,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.429363,48.208729
4,node,60648368,NaN,NaN,NaN,NaN,NaN,restaurant,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.349810,48.205262


In [76]:
def parse_input(text):
    """ """
    # Universit%C3%A4tsring+2,%201010+Wien
    data_json = requests.get(url=f'https://nominatim.openstreetmap.org/search?q={text}&format=json&polygon=1&addressdetails=1').json()
    lat = data_json[0]['lat']
    lon = data_json[0]['lon']
    geometry = gpd.GeoSeries.from_xy([lon], [lat], crs=local_utm_crs)
    df = pd.DataFrame({'Location':['Test']})
    gdf = gpd.GeoDataFrame(df, geometry = gpd.GeoSeries.from_xy([lon], [lat], crs=local_utm_crs))
    return gdf

def find_points_closeby(lat_lon, k = 500, max_distance = 500 ):
    results = tree.query((lat_lon), k = k, distance_upper_bound= max_distance)
    zipped_results = list(zip(results[0], results[1]))
    zipped_results = [i for i in zipped_results if i[0] != np.inf]
    return len(zipped_results)




df = parse_input('Universitätsring 2, 1010 Wien')
df[['longitude', 'latitude']] = df.apply(lambda x: get_lat_long(x['geometry']), axis=1)
df.head()

,Location,geometry,longitude,latitude
0,Test,POINT (16.361 48.210),16.361478,48.210337


In [85]:
t0 = time.time()
air_gdf = df.copy()
def get_tree(df):
    try:
        coords = list(zip(df.geometry.apply(lambda x: x.y).values,df.geometry.apply(lambda x: x.x).values))
        tree = spatial.KDTree(coords)
        return tree
    except Exception as e:
        print(e)
parameters = [restaurant, cafe , bar, station, biergarten, fast_food, pub, nightclub,theatre,university,attraction]
names = ['restaurant', 'cafe', 'bar', 'station', 'biergarten', 'fast_food', 'pub', 'nightclub','theatre','university','attraction']


air_gdf = gpd.GeoDataFrame(air_gdf, geometry = gpd.points_from_xy(air_gdf.longitude, air_gdf.latitude), crs = 4326)
air_gdf = air_gdf.to_crs(local_utm_crs)

for name, i in zip(names, parameters):
    tree = get_tree(i)
    #Apply the function
    air_gdf[name] = air_gdf.apply(lambda row: find_points_closeby((row.geometry.y, row.geometry.x)) , axis = 1)

print (f"Completed in {round(time.time() - t0)} s")
air_gdf.head()

Completed in 1 s


,Location,geometry,longitude,latitude,restaurant,cafe,bar,station,biergarten,fast_food,pub,nightclub,theatre,university,attraction
0,Test,POINT (601145.552 5340574.819),16.361478,48.210337,60,22,3,11,0,15,3,2,2,5,11


In [86]:
air_gdf = air_gdf.iloc[:, 4:-1]

air_gdf.head()

,restaurant,cafe,bar,station,biergarten,fast_food,pub,nightclub,theatre,university
0,60,22,3,11,0,15,3,2,2,5
